In [1]:
import pandas as pd
import numpy as np
from astroquery.simbad import Simbad
df = pd.read_csv('../csvs/gpi_campaign_list.csv')
df.head()

,star,ra,dec,comment,ra_deg,dec_deg
0,HIP 159,00:02:02.6,-02:45:58.2,H-coron (~1 hr),0.510833,-2.766167
1,HD 105,00:05:52.5,-41:45:11.0,H-coron (~1 hr),1.468750,-41.753056
2,HR 9,00:06:50.1,-23:06:27.1,H-coron (~1 hr); H-coron-pol,1.708750,-23.107528
3,HD 377,00:08:26,06:37:00,H-coron-pol; H-coron-pol,2.108333,6.616667
4,HIP 795,00:09:51.6,08:27:11.5,H-coron (~1 hr),2.465000,8.453194


In [2]:
customSimbad = Simbad()
customSimbad.remove_votable_fields('coordinates')
customSimbad.add_votable_fields('ra(2;A;ICRS;J2017.5;2000)', 'dec(2;D;ICRS;2017.5;2000)', 'sp', 'distance', 'diameter', 'flux(I)', 'flux(H)', 'fe_h')
customSimbad.get_votable_fields()

['main_id',
 'ra(2;A;ICRS;J2017.5;2000)',
 'dec(2;D;ICRS;2017.5;2000)',
 'sp',
 'distance',
 'diameter',
 'flux(I)',
 'flux(H)',
 'fe_h']

In [3]:
def query(row):
    res = customSimbad.query_object(row['star'])
    res = res.to_pandas()
    row = pd.concat([row, res.loc[0]])
    return row

In [4]:
%%time
dfs = []
dfs.append(df.loc[0:100].apply(query, axis=1))

CPU times: user 3.44 s, sys: 45.5 ms, total: 3.48 s
Wall time: 3.54 s


In [5]:
dfs.append(df.loc[100:200].apply(query, axis=1))

In [6]:
dfs.append(df.loc[200:300].apply(query, axis=1))

In [7]:
dfs.append(df.loc[300:400].apply(query, axis=1))

In [8]:
dfs.append(df.loc[400:500].apply(query, axis=1))

In [9]:
dfs.append(df.loc[500:600].apply(query, axis=1))

In [10]:
dfs.append(df.loc[600:700].apply(query, axis=1))

In [11]:
stellarpar = pd.concat(dfs)
stellarpar.tail()

,star,ra,dec,comment,ra_deg,dec_deg,MAIN_ID,RA_2_A_ICRS_J2017_5_2000,DEC_2_D_ICRS_2017_5_2000,SP_TYPE,...,Diameter_bibcode,FLUX_I,FLUX_H,Fe_H_Teff,Fe_H_log_g,Fe_H_Fe_H,Fe_H_flag,Fe_H_CompStar,Fe_H_CatNo,Fe_H_bibcode
626,HIP 117219,23:46:01.2,-40:10:56.9,H-coron (~1 hr),356.505000,-40.182472,b'HD 223011',23 46 01.4123,-40 10 57.234,b'A7III/IV',...,,NaN,5.879,7834,NaN,NaN,,b'SUN',,2012A&A...537A.120Z
627,HIP 117452,23:48:55.6,-28:07:49.0,H-coron (~1 hr); H-coron-pol,357.231667,-28.130278,b'* del Scl',23 48 55.6799,-28 07 48.974,b'A0Vnp',...,,4.550,4.643,9863,NaN,NaN,,b'SUN',,2006A&A...458..293P
628,HIP 117481,23:49:19.6,-27:51:14.9,H-coron (~1 hr); H-coron-pol,357.331667,-27.854139,b'HD 223408',23 49 19.4270,-27 51 15.328,b'F6V',...,,6.370,5.787,6255,4.18,NaN,,b'',,2003AJ....125.1598L
629,HIP 118008,23:56:10.7,-39:03:08.4,H-coron (~1 hr); H-coron-pol,359.044583,-39.052333,b'HD 224228',23 56 10.9830,-39 03 08.409,b'K2V',...,,7.194,6.007,0,NaN,NaN,,b'',,
630,HIP 118121,23:57:35.0,-64:17:53.1,H-coron (~1 hr),359.395833,-64.298083,b'* eta Tuc',23 57 35.2906,-64 17 53.622,b'A1V',...,,NaN,4.949,9057,NaN,NaN,,b'SUN',,2012A&A...537A.120Z


In [12]:
def bytes_to_string(row):
    row['S_Type'] = row['SP_TYPE'].decode('ascii')
    return row

stellarpar = stellarpar.apply(bytes_to_string, axis=1)

In [13]:
del stellarpar['SP_TYPE']

In [14]:
def splittype(row):
    try:
        row['Class'] = row["S_Type"][0]
        row['subtype'] = row["S_Type"][1]
    except:
        row['Class'] = np.nan
        row['subtype'] = np.nan
    return row

In [15]:
stellarpar = stellarpar[['S_Type', 'Distance_distance', 'FLUX_I', 'FLUX_H', 'Fe_H_Teff']]
stellarpar.tail()

,S_Type,Distance_distance,FLUX_I,FLUX_H,Fe_H_Teff
626,A7III/IV,67.1100,NaN,5.879,7834
627,A0Vnp,NaN,4.550,4.643,9863
628,F6V,35.3895,6.370,5.787,6255
629,K2V,21.9270,7.194,6.007,0
630,A1V,47.0754,NaN,4.949,9057


In [16]:
stellarpar = stellarpar.apply(splittype, axis=1)
stellarpar.tail()

,S_Type,Distance_distance,FLUX_I,FLUX_H,Fe_H_Teff,Class,subtype
626,A7III/IV,67.1100,NaN,5.879,7834,A,7
627,A0Vnp,NaN,4.550,4.643,9863,A,0
628,F6V,35.3895,6.370,5.787,6255,F,6
629,K2V,21.9270,7.194,6.007,0,K,2
630,A1V,47.0754,NaN,4.949,9057,A,1


In [17]:
# values from Habets & Heintze 1981 AA supplement series vol 46
def type_to_index(cls, subtype):
    d = {letter:num for num, letter in enumerate(['B','A','F','G','K','M'])}
    return 5*d[cls] + float(subtype)/2. # array is [B0, B2, B4, B6, B8, A0, A2, ..., M8]

def Mbol_to_L(M):
    return 10**(0.4*(4.74-M))

def get_Mbol(cls, subtype):
     Ms = [-6.49, -4.47, -2.7, -1.4, -0.1, 1.24, 1.64, 1.95, 2.28, 2.61, 2.95, 3.27, 3.58, 3.86, 4.12, 4.33, 4.54, 4.71, 4.9, 5.09, 5.31, 5.62, 6.02, 6.44, 6.88, 7.55, 8.69, 10.14, 0, 0]
     idx = type_to_index(cls, subtype)
     return 0.5*(Ms[int(np.floor(idx))] + Ms[int(np.ceil(idx))]) # return single value for integer index, average of before and after for fractional indices

def get_L(cls, subtype):
    if int(np.floor(idx)) < 5:
        # reroll 
        MLs = []

def get_T(cls, subtype):
    Ts = [29900, 22300, 17000, 14450, 12000, 9700, 9040, 8620, 8200, 7800, 7420, 7040, 6690, 6400, 6150, 5950, 5770, 5640, 5510, 5370, 5200, 4930, 4590, 4260, 3990, 3700, 3400, 3100, 2800, 2580]
    idx = type_to_index(cls, subtype)
    return 0.5*(Ts[int(np.floor(idx))] + Ts[int(np.ceil(idx))]) # return single value for integer index, average of before and after for fractional indices

def get_Mstar(cls, subtype):
    Ms = [15.6, 9.21, 6.0, 4.04, 2.84, 2.07, 1.94, 1.79, 1.66, 1.53, 1.4, 1.31, 1.23, 1.16, 1.09, 1.04, 0.986, 0.933, 0.879, 0.829, 0.781, 0.738, 0.693, 0.647, 0.585, 0.454, 0.262, 0.124, 0, 0]
    idx = type_to_index(cls, subtype)
    return 0.5*(Ms[int(np.floor(idx))] + Ms[int(np.ceil(idx))]) # return single value for integer index, average of before and after for fractional indices

def fill(row):
    cls = row['Class']
    subtype = row['subtype']
    
    Mbol = get_Mbol(cls, subtype)
    row['Mbol'] = Mbol
    row['Lum'] = Mbol_to_L(Mbol)
    row['Teff'] = get_T(cls, subtype)
    row['Mass'] = get_Mstar(cls, subtype)
    return row

# Drop NaNs

In [18]:
stellarpar = stellarpar[~stellarpar['Class'].isnull()]
stellarpar.shape

(626, 7)

# Manually drop remaining problematic rows

In [19]:
stellarpar = stellarpar.drop(262)
stellarpar = stellarpar.drop(336)
stellarpar = stellarpar.drop(343)
stellarpar = stellarpar.drop(424)
stellarpar = stellarpar.drop(530)
stellarpar = stellarpar.drop(576)
stellarpar.shape

(620, 7)

In [20]:
GPI = stellarpar.apply(fill, axis=1)

In [22]:
GPI.tail(20)

,S_Type,Distance_distance,FLUX_I,FLUX_H,Fe_H_Teff,Class,subtype,Mbol,Lum,Teff,Mass
610,G8V,48.0137,7.930,7.109,5611,G,8,5.090,0.724436,5370.0,0.8290
611,F6V,20.4599,NaN,4.606,6290,F,6,3.860,2.249055,6400.0,1.1600
612,G5V,30.4546,NaN,6.112,5603,G,5,4.805,0.941890,5575.0,0.9060
613,A2VpSrCrSi,48.9201,4.900,4.970,10228,A,2,1.640,17.378008,9040.0,1.9400
614,G8IV,63.1979,8.479,7.594,0,G,8,5.090,0.724436,5370.0,0.8290
615,M2Ve,15.9564,8.599,6.608,3481,M,2,8.690,0.026303,3400.0,0.2620
616,G1V,30.3128,NaN,5.738,5911,G,1,4.435,1.324342,5860.0,1.0130
617,K0IV,60.4511,NaN,7.504,0,K,0,5.310,0.591562,5200.0,0.7810
618,M0Ve,27.3734,8.646,6.770,3797,M,0,7.550,0.075162,3700.0,0.4540
619,K2V,34.0052,NaN,6.819,5229,K,2,5.620,0.444631,4930.0,0.7380


In [34]:
GPI.to_csv('../csvs/fullGPIsample.csv')